## 🧠 Aula: **Sistema de Usuários com Django — Teoria + Prática**

### 🎯 Objetivo da Aula:
Compreender e implementar um sistema de usuários no Django, com:
- Autenticação personalizada (usando e-mail como login)
- Cadastro, login, logout
- Associação de dados ao usuário autenticado (ex: tarefas)
- Proteção de rotas

---

## 🔍 Por que personalizar o modelo de usuário no Django?

### 🧩 O modelo de usuário padrão do Django:
O Django vem com um modelo de usuário pronto (`User`), que usa **username** como campo de login.

Mas, em sistemas modernos, é **muito comum usar o e-mail como identificador**. Também pode ser necessário adicionar campos personalizados, como `cpf`, `telefone`, `cargo`, etc.

---

## ✅ Quando criar um modelo de usuário personalizado?

- Você quer usar **e-mail como login**
- Precisa de **campos extras** no usuário
- Quer **mais controle** sobre a criação e validação de usuários

> 🔥 **Importante**: o modelo de usuário **deve ser personalizado no início do projeto**. Se fizer isso depois de aplicar migrações, pode quebrar o sistema.

---

## 🧱 Etapa 1: Criar um modelo de usuário customizado

### 📄 `userApp/models.py`

```python
from django.db import models
from django.contrib.auth.models import AbstractBaseUser, PermissionsMixin, BaseUserManager
```

### 🔧 Criando o gerenciador (`UserManager`)

Esse gerenciador define **como o usuário será criado**, tanto normal quanto superusuário.

```python
class UserManager(BaseUserManager):
    def get_by_natural_key(self, email):
        return self.get(email=email)

    def create_user(self, email, password=None, **extra_fields):
        if not email:
            raise ValueError("Usuário precisa de um e-mail.")
        email = self.normalize_email(email)
        user = self.model(email=email, **extra_fields)
        user.set_password(password)
        user.save()
        return user

    def create_superuser(self, email, password=None, **extra_fields):
        extra_fields.setdefault("is_staff", True)
        extra_fields.setdefault("is_superuser", True)
        return self.create_user(email, password, **extra_fields)
```

### 👤 Criando o modelo `User`

```python
class User(AbstractBaseUser, PermissionsMixin):
    email = models.EmailField(unique=True)
    name = models.CharField(max_length=50)
    is_active = models.BooleanField(default=True)
    is_staff = models.BooleanField(default=False)

    objects = UserManager()

    USERNAME_FIELD = 'email'
    REQUIRED_FIELDS = ['name']

    def __str__(self):
        return self.email
```

---

## ⚙️ Etapa 2: Informar ao Django que estamos usando nosso modelo

### 📄 `settings.py`

```python
AUTH_USER_MODEL = 'userApp.User'
```

---

## ⚙️ Etapa 3: Rodar as migrações

```bash
# Se for um projeto novo:
rm db.sqlite3
find . -path "*/migrations/*.py" -not -name "__init__.py" -delete
python manage.py makemigrations
python manage.py migrate
```

> 💡 Agora o banco entende que o modelo `User` é o novo padrão para login, grupos e permissões.

---

## 🧪 Etapa 4: Criar views de login, logout e cadastro

Essas views vão controlar o fluxo de autenticação.

### 📄 `userApp/views.py`

```python
from django.shortcuts import render, redirect
from django.contrib.auth import authenticate, login, logout, get_user_model
from django.contrib.auth.decorators import login_required

User = get_user_model()
```

### 🔑 Login

```python
def loginUser(request):
    if request.method == 'POST':
        email = request.POST.get('email')
        senha = request.POST.get('senha')
        user = authenticate(request, email=email, password=senha)
        if user:
            login(request, user)
            return redirect('viewTasks')
        return render(request, 'userApp/login.html', {'erro': 'Credenciais inválidas'})
    return render(request, 'userApp/login.html')
```

### 👤 Cadastro

```python
def registerUser(request):
    if request.method == 'POST':
        email = request.POST.get('email')
        nome = request.POST.get('name')
        senha = request.POST.get('senha')
        confirmar = request.POST.get('confirmar')

        if senha != confirmar:
            return render(request, 'userApp/register.html', {'erro': 'Senhas diferentes'})
        
        if User.objects.filter(email=email).exists():
            return render(request, 'userApp/register.html', {'erro': 'E-mail já cadastrado'})

        User.objects.create_user(email=email, password=senha, name=nome)
        return redirect('login')
    return render(request, 'userApp/register.html')
```

### 🚪 Logout

```python
def logoutUser(request):
    logout(request)
    return redirect('login')
```

---

## 🌐 Etapa 5: Configurar rotas

### 📄 `core/urls.py`

```python
from django.contrib import admin
from django.urls import path, include
from userApp import views as user_views

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', user_views.loginUser, name='login'),
    path('logout/', user_views.logoutUser, name='logout'),
    path('register/', user_views.registerUser, name='register'),
    path('toDoList/', include('toDoListApp.urls')),
]
```

---

## 📦 Etapa 6: Associar tasks ao usuário logado

### 📄 `toDoListApp/models.py`

```python
from django.db import models
from django.conf import settings

class Task(models.Model):
    user = models.ForeignKey(settings.AUTH_USER_MODEL, on_delete=models.CASCADE)
    nameTask = models.CharField(max_length=100)
    description = models.TextField()
    dateCreation = models.DateTimeField(auto_now_add=True)
```

---

## 👀 Etapa 7: Mostrar apenas as tasks do usuário logado

### 📄 `toDoListApp/views.py`

```python
@login_required(login_url='login')
def viewTasks(request):
    name = request.GET.get('nameTask')
    if name:
        tasks = Task.objects.filter(user=request.user, nameTask__icontains=name)
    else:
        tasks = Task.objects.filter(user=request.user)
    return render(request, 'toDoListApp/viewTasks.html', {'tasks': tasks})
```

---

## ✅ Conclusão da Aula

### O que aprendemos:
- Quando e por que personalizar o modelo de usuário
- Como criar um modelo de autenticação por e-mail
- Como configurar login, logout e cadastro
- Como proteger rotas com `@login_required`
- Como associar dados ao usuário autenticado